In [1]:
import os
import psycopg2
import configparser
import pandas as pd
import numpy as np
import config
import json

import shioaji as sj
import time
import datetime

In [2]:

filepath = os.getcwd() + r'\Stock_SQL'

INITXT=filepath + r"\config.ini"
config = configparser.ConfigParser()
config.read(INITXT)
host    = config.get("POSTGRES","host")
user    = config.get("POSTGRES","user")
passwd  = config.get("POSTGRES","passwd")
db      = config.get("POSTGRES","db")

# list(config['POSTGRES'])
# list(config)
# host = config['POSTGRES']['host']

In [3]:
api = sj.Shioaji()

api.login(
        person_id='M122548082',
        passwd='Rs781023'
)

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


[StockAccount(person_id='M122548082', broker_id='9A9W', account_id='0351577', signed=True, username='江晉毅\u3000\u3000')]

In [6]:
def Json_DT(stock_number,Date):
    Stock_API = api.Contracts.Stocks[stock_number]
    Tick = api.ticks(Stock_API,Date)
    Tick_Data_DF = pd.DataFrame({**Tick})
    Tick_Data_DF.ts = pd.to_datetime(Tick_Data_DF.ts)
    
    Tick_Data_DF = Tick_Data_DF[['close','tick_type','volume']]
    
    Tick_Data_DF['outer_buy'] = 0
    Tick_Data_DF['inner_sell'] = 0
    Tick_Data_DF['middle_price'] = 0
    
    Tick_Data_DF['middle_price'] = np.where(Tick_Data_DF['tick_type']==0,Tick_Data_DF['volume'],0)
    Tick_Data_DF['outer_buy'] = np.where(Tick_Data_DF['tick_type']==1,Tick_Data_DF['volume'],0)
    Tick_Data_DF['inner_sell'] = np.where(Tick_Data_DF['tick_type']==2,Tick_Data_DF['volume'],0)
    
    Tick_Data_DF = Tick_Data_DF[['close','volume','outer_buy','inner_sell','middle_price']]
    
    Tick_Data_DF = Tick_Data_DF.groupby('close').sum().sort_index(ascending=False)
    
    Tick_Data_DF = Tick_Data_DF.to_json(orient='split')
    Tick_Data_DF = {'Data_Date':Date,"Data_Detail":Tick_Data_DF}
    
    Tick_Data_DF = json.dumps(Tick_Data_DF,indent=4) 

    return(Tick_Data_DF)

In [7]:
try:
    connection = psycopg2.connect(user=user,
                              password=passwd,
                              host=host,
                              port="5432",
                              database='stock')

    cursor = connection.cursor()

    All_Table_SQL_Message  = ''' select * from pg_tables where schemaname='public'; '''

    SQL_Execute  = cursor.execute(All_Table_SQL_Message)
    
    All_Table_List = np.array(cursor.fetchall())
    
    All_Table_List = All_Table_List[:,1]
    
    All_Table_List = np.delete(All_Table_List,np.argwhere(All_Table_List=='tse'))
    All_Table_List = np.delete(All_Table_List,np.argwhere(All_Table_List=='otc'))
    All_Table_List = np.delete(All_Table_List,np.argwhere(All_Table_List=='time_list'))
    
    #######################################################
    
    Time_List_SQL_Message  = ''' select * from time_list; '''
    
    SQL_Execute  = cursor.execute(Time_List_SQL_Message)
    
    Time_List = np.array(cursor.fetchall())
    
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if connection:
        cursor = connection.cursor()
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed


In [8]:
try:
    connection = psycopg2.connect(user=user,
                              password=passwd,
                              host=host,
                              port="5432",
                              database='stock')

    cursor = connection.cursor()
    
    for i in range(All_Table_List.shape[0]):
        
        Stock_Number = str(All_Table_List[i][4:])
        
        All_Table_SQL_Message  = ''' select time_date from {}; '''.format(All_Table_List[i])

        SQL_Execute  = cursor.execute(All_Table_SQL_Message)
    
        Table_Data = np.array(cursor.fetchall())
        
        Missing_Data = np.setdiff1d(Time_List,Table_Data)
        
        ########################################
        
        Json_Data = np.array([])
        
        for x in range(Missing_Data.shape[0]):
            
            json_time = str(Missing_Data[x])
            
            json_data = Json_DT(Stock_Number,json_time)
  
            json_time_data = np.array([Missing_Data[x],json_data])
            
            Json_Data = np.append(Json_Data,json_time_data)
            
        Json_Data = Json_Data.reshape(Missing_Data.shape[0],2)

        ########################################

        Insert_SQL_Message  = ''' insert into {} values '''.format(All_Table_List[i])
        
        for x in range(Json_Data.shape[0]):
            Insert_SQL_Message = Insert_SQL_Message + '''('{}','{}'),'''.format(str(Json_Data[x][0]),Json_Data[x][1])
            
        Insert_SQL_Message = Insert_SQL_Message[:-1]+';'
        SQL_Execute  = cursor.execute(Insert_SQL_Message)
        connection.commit()
        print(Insert_SQL_Message)

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if connection:
        cursor = connection.cursor()
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

 insert into tse_1103 values ('2021-12-28','{
    "Data_Date": "2021-12-28",
    "Data_Detail": "{\"columns\":[\"volume\",\"outer_buy\",\"inner_sell\",\"middle_price\"],\"index\":[20.7,20.65,20.6,20.55,20.5],\"data\":[[75,59,16,0],[149,104,45,0],[66,17,49,0],[107,73,34,0],[101,0,91,10]]}"
}'),('2021-12-29','{
    "Data_Date": "2021-12-29",
    "Data_Detail": "{\"columns\":[\"volume\",\"outer_buy\",\"inner_sell\",\"middle_price\"],\"index\":[20.85,20.8,20.75,20.7],\"data\":[[42,42,0,0],[70,69,1,0],[268,209,59,0],[52,0,48,4]]}"
}'),('2021-12-30','{
    "Data_Date": "2021-12-30",
    "Data_Detail": "{\"columns\":[\"volume\",\"outer_buy\",\"inner_sell\",\"middle_price\"],\"index\":[20.8,20.75,20.7,20.65],\"data\":[[19,0,19,0],[190,131,59,0],[152,12,140,0],[13,0,13,0]]}"
}'),('2021-12-31','{
    "Data_Date": "2021-12-31",
    "Data_Detail": "{\"columns\":[\"volume\",\"outer_buy\",\"inner_sell\",\"middle_price\"],\"index\":[],\"data\":[]}"
}');
 insert into otc_1240 values ('2021-12-28','{
 